In [1]:
import sys
import os
import time
import jpype
import jpype.imports
import json

In [2]:
!pwd
#set ATALA_PRISM_JARS = /Users/alex/Projects/rootid/didcomm-mediator/prism-cli-v1.4.1/lib
!export ATALA_PRISM_JARS=/Users/alex/Projects/rootid/didcomm-mediator/prism-cli-v1.4.1/lib

/Users/alex/Projects/rootid/didcomm-mediator/sample-notebooks


In [3]:
try:
    sdk_gradle_home = '/Users/alex/Projects/rootid/didcomm-mediator/prism-cli-v1.4.1/lib'
except KeyError:
    print("ERROR: `ATALA_PRISM_JARS` variable is not set.")
    print("Please, set it to the directory with Atala PRISM SDK dependencies JARs.")
    sys.exit(1)

In [4]:
try:
    jpype.imports.registerDomain('sdk', alias='io')
except ImportError as err:
    print(err.msg)
    sys.exit(ERROR)
jpype.startJVM(
    classpath=[
        os.path.join(sdk_gradle_home, f) for f in os.listdir(sdk_gradle_home)
            if f.endswith('.jar')
    ]
)

In [5]:
#get docstring from class
print(dir(ECPublicKey))

NameError: name 'ECPublicKey' is not defined

In [6]:
from sdk.iohk.atala.prism.protos import *
from sdk.iohk.atala.prism.api.node import *
from sdk.iohk.atala.prism.api.models import *
from sdk.iohk.atala.prism.api import *
from sdk.iohk.atala.prism.crypto.derivation import *
from sdk.iohk.atala.prism.crypto.keys import *
from sdk.iohk.atala.prism.identity import *
from kotlinx.serialization.json import *
from kotlinx.serialization import DeserializationStrategy

KeyGenerator = KeyGenerator.INSTANCE
KeyDerivation = KeyDerivation.INSTANCE
MasterKeyUsage = MasterKeyUsage.INSTANCE
IssuingKeyUsage = IssuingKeyUsage.INSTANCE
RevocationKeyUsage = RevocationKeyUsage.INSTANCE
AuthenticationKeyUsage = AuthenticationKeyUsage.INSTANCE
AtalaOperationStatus = AtalaOperationStatus.INSTANCE
PrismDID = PrismDid

In [7]:
def wait_until_confirmed(node_api: NodePublicApi, operation_id: AtalaOperationId):
    """Waits until operation is confirmed by Cardano network
    
    Confirmation doesn't necessarily mean that operation was applied.
    For example, it could be rejected because of an incorrect signature or other reasons.

    :param node_api: Atala PRISM Node API object
    :type node_api: NodePublicApi
    :param operation_id: Atala PRISM operation ID
    :type operation_id: AtalaOperationId
    """
    operation_status = int(node_api.getOperationInfo(operation_id).join().getStatus())
    while operation_status != AtalaOperationStatus.getCONFIRMED_AND_APPLIED() \
        and operation_status != AtalaOperationStatus.getCONFIRMED_AND_REJECTED():
        print(f"Current operation status: {AtalaOperationStatus.asString(operation_status)}")
        time.sleep(1)
        operation_status = int(node_api.getOperationInfo(operation_id).join().getStatus())

def prepare_keys_from_mnemonic(mnemonic: MnemonicCode, password: str):
    """Creates a map of potentially useful keys out of a mnemonic code

    :param mnemonic: Mnemonic to generate keys from
    :type mnemonic: MnemonicCode
    :param password: Password for seed generation
    :type password: str
    :return: Map of keys
    :rtype: dict<String, ECKeyPair>
    """
    did_key_map = {}
    seed = KeyDerivation.binarySeed(mnemonic, password)
    did_key_map[PrismDid.getDEFAULT_MASTER_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, MasterKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_ISSUING_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, IssuingKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_REVOCATION_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, RevocationKeyUsage, 0)
    return did_key_map

def prepare_keys_from_seed(seed: bytes):
    """Creates a map of potentially useful keys out of a mnemonic code

    :param mnemonic: Mnemonic to generate keys from
    :type mnemonic: MnemonicCode
    :param password: Password for seed generation
    :type password: str
    :return: Map of keys
    :rtype: dict<String, ECKeyPair>
    """
    did_key_map = {}
    did_key_map[PrismDid.getDEFAULT_MASTER_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, MasterKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_ISSUING_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, IssuingKeyUsage, 0)
    did_key_map[PrismDid.getDEFAULT_REVOCATION_KEY_ID()] = KeyGenerator.deriveKeyFromFullPath(seed, 0, RevocationKeyUsage, 0)
    return did_key_map

In [8]:
environment = "ppp-vasil.atalaprism.io"
node_auth_api = NodeAuthApiAsyncImpl(GrpcOptions("http", environment, 50053))

In [9]:
#print code for GrpcOptions class
print(GrpcOptions.__doc__)


Java class 'io.iohk.atala.prism.protos.GrpcOptions'

    Extends:
        java.lang.Object

  Constructors:
    * GrpcOptions(java.lang.String, java.lang.String, int)
    * GrpcOptions(java.lang.String, java.lang.String, int, java.lang.String)
    * GrpcOptions(java.lang.String, java.lang.String, int, java.lang.String, int, kotlin.jvm.internal.DefaultConstructorMarker)



## Generating a DID

In [10]:
print(PrismDid.getDEFAULT_MASTER_KEY_ID())
print("Issuer: Generates and registers a DID")
issuer_keys = prepare_keys_from_mnemonic(KeyDerivation.randomMnemonicCode(), "passphrase")
issuer_unpublished_did = PrismDid.buildLongFormFromMasterPublicKey(
    issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPublicKey()
)
issuer_did = issuer_unpublished_did.asCanonical()
print(issuer_did.toString())
node_payload_generator = NodePayloadGenerator(
    issuer_unpublished_did,
    {
        PrismDid.getDEFAULT_MASTER_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey(),
        PrismDid.getDEFAULT_ISSUING_KEY_ID(): issuer_keys[PrismDid.getDEFAULT_ISSUING_KEY_ID()].getPrivateKey()
    },
    
)

print(issuer_unpublished_did)
print(PrismDid.getDEFAULT_MASTER_KEY_ID())
print(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey())
issuer_create_did_info = node_payload_generator.createDid(PrismDid.getDEFAULT_MASTER_KEY_ID())
issuer_create_did_operation_id = node_auth_api.createDid(
    issuer_create_did_info.getPayload(),
    issuer_unpublished_did,
    PrismDid.getDEFAULT_MASTER_KEY_ID()
).join()

print(f"""
- Issuer sent a request to create a new DID to PRISM Node.
- The transaction can take up to 10 minutes to be confirmed by the Cardano network.
- Operation identifier: {issuer_create_did_operation_id.hexValue()}
""")
create_did_operation_result = wait_until_confirmed(node_auth_api, issuer_create_did_operation_id)

print(f"- DID with id {issuer_did} is created")

master0
Issuer: Generates and registers a DID


[main] INFO org.bitcoinj.crypto.MnemonicCode - PBKDF2 took 146.1 ms


did:prism:1b82892792b6d2cfef15749867b492e942de1fb2746629da82ef22b3a6a31bdb
did:prism:1b82892792b6d2cfef15749867b492e942de1fb2746629da82ef22b3a6a31bdb:Cj8KPRI7CgdtYXN0ZXIwEAFKLgoJc2VjcDI1NmsxEiECdetMOAf8sh5r9pg151SV7KNazA6Bi2U-32EiSaSaBn0
master0
io.iohk.atala.prism.crypto.keys.ECPrivateKey@7fda2001

- Issuer sent a request to create a new DID to PRISM Node.
- The transaction can take up to 10 minutes to be confirmed by the Cardano network.
- Operation identifier: 0ad03187cca7130d6d8d0bcb68e77711f09d309f8f3c79066110e91c25989b22

Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operation status: PENDING_SUBMISSION
Current operatio

In [57]:
print(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded())
print(type(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey()))
bint = int(str(issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded()),16)


7d96e4807f9a6ce6ce39b52f9fa602797268eb2437d4d52203d7b96fbfee7986
<java class 'io.iohk.atala.prism.crypto.keys.ECPrivateKey'>
io.iohk.atala.prism.crypto.keys.ECPublicKey@2a50b32d


In [19]:
key = issuer_keys[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey()
# print(key.getCurvePoint().getX().getCoordinate())
# print(key.getCurvePoint().getY().getCoordinate())
print(dir(key))
print(key.getHexEncoded())

['DefaultImpls', '__class__', '__del__', '__delattr__', '__dict__', '__dir__', '__doc__', '__eq__', '__format__', '__ge__', '__getattribute__', '__gt__', '__hash__', '__init__', '__init_subclass__', '__le__', '__lt__', '__module__', '__ne__', '__new__', '__reduce__', '__reduce_ex__', '__repr__', '__setattr__', '__sizeof__', '__str__', '__subclasshook__', '__weakref__', 'equals', 'getClass', 'getD', 'getD$prism_crypto', 'getEncoded', 'getHexEncoded', 'getKey$prism_crypto', 'hashCode', 'notify', 'notifyAll', 'toString', 'wait']
a765343d4965c02862070c623ad1a1aa803928049c511eb37785c0e0be260c17


In [103]:
#from hex string to bytes
st = '049dbd906bb386ba7d66beebf5ec3a0e5c7c5fd940beca50b0c4f03bbefdd987283d40ffe2e0a1ba652e297db72222fc789c5e7095470868898832cccc8daa952d'
b = bytes.fromhex(st)
base58.b58encode(b)

b'7r8orddYZTNLN8WGkY51aVsS5dRc9KkpM7TAFTxmEQsr9Wdq4FLeZNFWEhsVaouWnNwPjyntKE81zhhkHch368g1xdJC'

Error: Incorrect padding

In [128]:
pri(key.getD()
)

b'6DteJRfAUmTzpCqUriSnxRFtfy25dJ2T4RtWVrzekWJqQax'

In [136]:
#remove empty bytes
bbb= int.to_bytes(int_, 58, 'big').lstrip(b'\x00')

In [159]:
keys = {"privateKey": "f554d07ad956ccd91a08ee1d9882749518fb131715c8f30139ff8d55d4725add",
    "publicKey": "049dbd906bb386ba7d66beebf5ec3a0e5c7c5fd940beca50b0c4f03bbefdd987283d40ffe2e0a1ba652e297db72222fc789c5e7095470868898832cccc8daa952d",
}
#from hex string to int 
int_ = int(keys["privateKey"], 16)
print(int_)
print(len(str(int_)))
#remove empty bytes
bbb= int.to_bytes(int_, 58, 'big').lstrip(b'\x00')


110966501931583750082985051953930554202289874904595929169354067567179616901853
78


In [163]:
#pytohn base58 decode 
import base58
bdecoded = base58.b58decode('231cRx1fhyNzrdj9i3UseKm1ApgMwyDLbKtJJH5AacEwL')
sol = int.from_bytes(bdecoded, byteorder='big')
print(sol)
print(len(str(sol)))

403302766519217024866501033698194361114496144622136351637745117680854454412363
78


In [67]:
len('zQ3shokFTS3brHcDQrn82RUDfCZESWL1ZdCEJwekUDPQiYBme')

49

In [75]:
base58.b58encode(b'\xe7\x01' + bdecoded[2:])

b'Q3shnxmSoA9BJ2Djspq8RZkh9MNcUSYvFmP8Fp46aQqhpio4'

In [71]:
#check if the first bytes is 0x5820
publi_key_b58 = base58.b58encode(bdecoded[2:])

In [76]:
403302766519217024866501033698194361114496144622136351637745117680854454412365

403302766519217024866501033698194361114496144622136351637745117680854454412365

In [85]:
#int to bytes
#remove empty bytes
base58.b58encode(b'\xe7\x01' + int.to_bytes(403302766519217024866501033698194361114496144622136351637745117680854454412363, 58, 'big').lstrip(b'\x00'))

b'Q3shnxmSoA9BJ2Djspq8RZkh9MNcUSYvFmP8Fp46aQqhpio4'

In [139]:
len(int.to_bytes(403302766519217024866501033698194361114496144622136351637745117680854454412363, 58, 'big').lstrip(b'\x00'))

33

In [125]:
# Issuer generates a credential to Holder identified by its unpublished DID
holder_did = PrismDid.fromString("did:prism:478c2a9e906b812531b606624ebd9e5a263ea62be5cdef78d69165976b736bad")
credential_claim = CredentialClaim(
    holder_did,
    JsonObject(
        {
            "name": JsonLiteral("Jose Lopez Portillo", True),
            "certificate": JsonLiteral("Certificate of PRISM SDK Python tutorial completion", True)
        }
    )
)
# node_payload_generator = NodePayloadGenerator(
#     issuer_unpublished_did,
#     {
#         PrismDid.getDEFAULT_MASTER_KEY_ID(): stored_object[PrismDid.getDEFAULT_MASTER_KEY_ID()],
#         PrismDid.getDEFAULT_ISSUING_KEY_ID(): stored_object[PrismDid.getDEFAULT_ISSUING_KEY_ID()],
#     },
    
# )


issue_credential_info = node_payload_generator.issueCredentials(
    PrismDid.getDEFAULT_ISSUING_KEY_ID(),
    [credential_claim]
)

In [127]:
#issuer_did2 = PrismDid.fromString(stored_object["did"])
issue_credential_batch_operation_id = node_auth_api.issueCredentials(
    issue_credential_info.getPayload(),
    issuer_unpublished_did.asCanonical(),
    PrismDid.getDEFAULT_ISSUING_KEY_ID(),
    issue_credential_info.getMerkleRoot()
).join()

issue_credential_batch_operation_result = \
    wait_until_confirmed(node_auth_api, issue_credential_batch_operation_id)

Current operation status: PENDING_SUBMISSION


In [130]:
holder_signed_credential = issue_credential_info.getCredentialsAndProofs()[0].getSignedCredential()
holder_credential_merkle_proof = issue_credential_info.getCredentialsAndProofs()[0].getInclusionProof()

print(f"""
Issuer [{issuer_did}] issued new credentials for the holder [{holder_did.asCanonical()}].
- issueCredentialBatch operation identifier: {issue_credential_batch_operation_id.hexValue()}
- Credential content: {holder_signed_credential.getContent()}
- Signed credential: {holder_signed_credential.getCanonicalForm()}
- Inclusion proof (encoded): {holder_credential_merkle_proof.encode()}
- Batch id: {issue_credential_info.getBatchId()}"""
)


Issuer [did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0] issued new credentials for the holder [did:prism:478c2a9e906b812531b606624ebd9e5a263ea62be5cdef78d69165976b736bad].
- issueCredentialBatch operation identifier: 4835ca1123ce314e8b1f38d7c991feecc5ed358b901fd5bf8e58799541f34b83
- Credential content: CredentialContent(fields={"id":"did:prism:c57dd70074a1cac4741e63209248288e1fb9ffcd8887bf731e0ae76c42b078b0","keyId":"issuing0","credentialSubject":{"name":"Jose Lopez Portillo","certificate":"Certificate of PRISM SDK Python tutorial completion","id":"did:prism:478c2a9e906b812531b606624ebd9e5a263ea62be5cdef78d69165976b736bad"}})
- Signed credential: eyJpZCI6ImRpZDpwcmlzbTpjNTdkZDcwMDc0YTFjYWM0NzQxZTYzMjA5MjQ4Mjg4ZTFmYjlmZmNkODg4N2JmNzMxZTBhZTc2YzQyYjA3OGIwIiwia2V5SWQiOiJpc3N1aW5nMCIsImNyZWRlbnRpYWxTdWJqZWN0Ijp7Im5hbWUiOiJKb3NlIExvcGV6IFBvcnRpbGxvIiwiY2VydGlmaWNhdGUiOiJDZXJ0aWZpY2F0ZSBvZiBQUklTTSBTREsgUHl0aG9uIHR1dG9yaWFsIGNvbXBsZXRpb24iLCJpZCI6ImRpZDpwcmlzbTo0

In [158]:
print(str(issue_credential_info.getBatchId().getId()))

67f0609570266a51305a731c4198ff07f89e7e1d13b0711952cf0d45e42d274f


In [29]:
# Verifier, who owns credentialClam, can easily verify the validity of the credentials
credential_verification_result = node_auth_api.verify(
    holder_signed_credential,
    holder_credential_merkle_proof
).join()

verification_errors = credential_verification_result.getVerificationErrors()
assert list(verification_errors) == [], "VerificationErrors should be empty"

AssertionError: VerificationErrors should be empty

In [83]:
mnemonic = KeyDerivation.randomMnemonicCode()
phrase = "password"
seed = bytes(KeyDerivation.binarySeed(mnemonic, phrase))

[main] INFO org.bitcoinj.crypto.MnemonicCode - PBKDF2 took 21.20 ms


In [84]:
print(type(seed))

<class 'bytes'>


In [85]:
keys1 = prepare_keys_from_mnemonic(mnemonic, phrase)

[main] INFO org.bitcoinj.crypto.MnemonicCode - PBKDF2 took 22.36 ms


In [86]:
keys2 = prepare_keys_from_seed(seed)

In [87]:
print(keys1[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded())

2447d8ab16457177306821ac64a7c49eb2a1b2f3d472b97dd42bcc3d7fd8481f


In [88]:
print(keys2[PrismDid.getDEFAULT_MASTER_KEY_ID()].getPrivateKey().getHexEncoded())

2447d8ab16457177306821ac64a7c49eb2a1b2f3d472b97dd42bcc3d7fd8481f


In [107]:
cl = JsonObject(
        {
            "name": JsonLiteral("Jose Lopez Portillo", True),
            "certificate": JsonLiteral("Certificate of PRISM SDK Python tutorial completion", True)
        }
    )
jsonString = '{"firstName":"Tom", "lastName": "Broody"}'

In [108]:
print(type(cl))

<java class 'kotlinx.serialization.json.JsonObject'>
